In [1]:
import numpy as np
import pandas as pd
import scanpy as sc

#### selected dataset - Single-Cell RNA-Sequencing From Mouse Incisor Reveals Dental Epithelial Cell-Type Specific Genes

In [15]:
input_file = 'data/GSM4407907/'
adata = sc.read_10x_mtx(
    input_file,  # the directory with the `.mtx` file
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True)                              # write a cache file for faster subsequent reading

### filtering

In [16]:
adata

AnnData object with n_obs × n_vars = 6260 × 48816
    var: 'gene_ids'

In [17]:
sc.pp.filter_cells(adata, min_genes=10)
sc.pp.filter_genes(adata, min_cells=10)

In [18]:
adata

AnnData object with n_obs × n_vars = 6260 × 18021
    obs: 'n_genes'
    var: 'gene_ids', 'n_cells'